# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



In [21]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [111]:
def global_alignment(source, target, voxel_size=0.05, corr_length=0.9,
                     radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                     ransac_stopping_criteria=100000, dont_downsample_source_mode=False):
    #####################################
    # Downsample and find features here #
    #####################################
    # 1 Downsample both point clouds
    if dont_downsample_source_mode:
        source_down = source
        target_down = target.voxel_down_sample(voxel_size=voxel_size)
    else:
        source_down = source.voxel_down_sample(voxel_size=voxel_size)
        target_down = target.voxel_down_sample(voxel_size=voxel_size)

    # 2 Estimate surface normals of pointclouds
    #source_down.estimate_normals()
    #target_down.estimate_normals()
    radius_feature = voxel_size * radius_scale
    source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))
    target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))

    # 3 FPFH feature descriptor and correspondance on the downsampled pointcloud. 
    # Raius used to describe a feature
    
    fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(source_down,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))

    fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(target_down,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))

    ##########
    # RANSAC #
    ##########
    # We can try both point-to-point or point-to plane
    point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    distance_threshold = voxel_size * distance_threshold_scale
    c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(normal_checker)

    checker_list = [c0,c1,c2]
    
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down,
        target_down,
        fpfh_source, 
        fpfh_target,
        True, # mutual_filter
        voxel_size*2,
        point_to_point, # point_to_point or point_to_plane
        checkers = checker_list,
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(ransac_stopping_criteria, 0.999)
        )
    return result, source_down, target_down

In [80]:
r1 = o3d.io.read_point_cloud("global_registration/r1.pcd")
r2 = o3d.io.read_point_cloud("global_registration/r2.pcd")
ransac_result_r1_r2,_,_ = global_alignment(r1, r2, 
                    voxel_size=0.05, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=100000)

draw_registrations(r1, r2, ransac_result_r1_r2.transformation, True)

In [81]:
r1 = o3d.io.read_point_cloud("global_registration/car1.ply")
r2 = o3d.io.read_point_cloud("global_registration/car2.ply")

#draw_registrations(r1, r2, recolor=True)

ransac_result_r1_r2,_,_ = global_alignment(r1, r2, 
                    voxel_size=0.05, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=1000000)

draw_registrations(r1, r2, ransac_result_r1_r2.transformation, True)



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [110]:
def preprocess(i, step, path="car_challange"):
    """Loads images, converts them to rgbd and transform into point cloud"""
    #######preprocess images
    ####load images
    color_raw0 = o3d.io.read_image(path+"/rgb/000%04d.jpg" %i)
    depth_raw0 = o3d.io.read_image(path+"/depth/000%04d.png" %i)


    ### rgbd colour
    rgbd_image0 = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw0, 
        depth_raw0, 
        convert_rgb_to_intensity = True)

    #### Point cloud
    camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

    point_cloud = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image0, camera)


    # Flip it, otherwise the pointcloud will be upside down
    point_cloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    return point_cloud #extend our point cloud
      

In [136]:
def full_allignment(nb_images, step, threshold, trans_init, path="car_challange",
                    voxel_size=0.05, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=1000000, dont_downsample_source_mode=True):

    sample_batches = []
    global_transformations = []
    local_transformations = []
    n=10

    for i in range (2, nb_images, step):
        print("loaded imag", i, "\r")
        if (i == 2):
            source = preprocess(1, step, path=path)
            print("i = 1, loaded:", source)
            # only downsample source once
            source = source.voxel_down_sample(voxel_size=voxel_size)
        else:
            source = new_source

        if dont_downsample_source_mode and (i//step)%n == 0 and i != 2:
            print("Downsampling! Wohooo :D")
            print(i, (i//step), (i//step)%n)
            sample_batches.append(copy.deepcopy(new_source))

            #source = preprocess(i-step, step, path=path)
            source = source.voxel_down_sample(voxel_size=voxel_size)

        target = preprocess(i, step, path) 
        
        #####################
        # Global allignment #
        #####################

        global_ransac_result,reduced_source,reduced_target = global_alignment(source, target, 
                    voxel_size=voxel_size, corr_length=corr_length, radius_scale=radius_scale,
                     max_nn=max_nn, distance_threshold_scale=distance_threshold_scale, normal_checker=normal_checker,
                    ransac_stopping_criteria=ransac_stopping_criteria, dont_downsample_source_mode=dont_downsample_source_mode)
        
        global_transformations.append(global_ransac_result.transformation)
        # Globally allign
        reduced_source = reduced_source.transform(global_ransac_result.transformation)

        ####################
        # Local allignment #
        ####################
        # ICP is local alignment for further improvements. 
        point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

        icp_result = o3d.pipelines.registration.registration_icp(
            reduced_source, reduced_target, threshold, trans_init,
            point_to_plane)
        local_transformations.append(icp_result.transformation)

        # Add new locally alligned points
        new_source = reduced_source.transform(icp_result.transformation) + reduced_target

    if dont_downsample_source_mode: # do some stitching!
        for k, pc in enumerate(sample_batches):
            
            # Apply all transformation that happened later on
            for (global_trans, local_trans) in zip(global_transformations[(k+1)*n + 1:], local_transformations[(k+1)*n + 1:]):
                pc = pc.transform(global_trans)
                pc = pc.transform(local_trans)

            if k==0:
                new_source = copy.deepcopy(pc)
            else:
                new_source += pc

    return new_source, reduced_target, icp_result, sample_batches


#### Two modes:

1. dont_downsample_source_mode=False: now we will downsample source and target every iteration. Problem: the car gets really grainy.
2.  dont_downsample_source_mode=True: Now we downsample the source every nth image: the parts of the car we haven't seen in a while will be more grainy :(

In [137]:

nb_images = 1000 #max 400
step = 5 #must be devidible by 400: 1, 2, 4, 5, 8, 10, 20...
voxel_size = 0.05
# Parameters
threshold = 0.02
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
# source = o3d.geometry.PointCloud() #initialize point cloud

new_source, reduced_target,icp_result, sample_batches  = full_allignment(
                    nb_images, step, threshold, trans_init, path="car_challange",
                    voxel_size=voxel_size, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=1000000, dont_downsample_source_mode=True)

draw_registrations(new_source, reduced_target, icp_result.transformation, True)


loaded imag 2 
i = 1, loaded: PointCloud with 166218 points.
loaded imag 7 
loaded imag 12 
loaded imag 17 
loaded imag 22 
loaded imag 27 
loaded imag 32 
loaded imag 37 
loaded imag 42 
loaded imag 47 
loaded imag 52 
Downsampling! Wohooo :D
52 10 0
loaded imag 57 
loaded imag 62 
loaded imag 67 
loaded imag 72 
loaded imag 77 
loaded imag 82 
loaded imag 87 
loaded imag 92 
loaded imag 97 
loaded imag 102 
Downsampling! Wohooo :D
102 20 0
loaded imag 107 
loaded imag 112 
loaded imag 117 
loaded imag 122 
loaded imag 127 
loaded imag 132 
loaded imag 137 
loaded imag 142 
loaded imag 147 
loaded imag 152 
Downsampling! Wohooo :D
152 30 0
loaded imag 157 
loaded imag 162 
loaded imag 167 
loaded imag 172 
loaded imag 177 
loaded imag 182 
loaded imag 187 
loaded imag 192 
loaded imag 197 
loaded imag 202 
Downsampling! Wohooo :D
202 40 0
loaded imag 207 
loaded imag 212 
loaded imag 217 
loaded imag 222 
loaded imag 227 
loaded imag 232 
loaded imag 237 
loaded imag 242 
loaded imag 

In [73]:
draw_registrations(new_source, reduced_target, icp_result.transformation, True)


In [132]:
for pc in sample_batches:
    o3d.visualization.draw_geometries([pc])

In [ ]:
o3d.io.write_point_cloud("Final_results.pcd", new_source)